In [160]:
import cv2
import re
import numpy as np
from PIL import Image
from pyzbar import pyzbar
from cnocr import CnOcr

#发票类
class Invoice:
    def __init__(self,payer='',money='',date=''):
        self.payer=payer
        self.money=money
        self.date=date
    # 交易主体、金额、日期
    payer: str
    money:str
    date:str

#检查是否符合日期格式
def isDate(string):
    date_pattern = r"\d{4}年\d{2}月\d{2}日"
    if re.match(date_pattern, string):
        return True
    else:
        return False
        
def ocr_getinfo(index):
    image_path = 'C:\\Users\\Max\\Desktop\\发票-大数据集\\发票-大数据集\\b\\b'+str(index)+'.jpg'
    #旋转图片
    
    
    ocr = CnOcr()
    out = ocr.ocr(image_path)
    payer = '深圳市购机汇网络有限公司'
    
    invoice = Invoice()
    
    # 调试用：在图中画出识别的范围并保存
#     image = cv2.imread(image_path)
#     for item in out:
#         position = item['position']
#         for i in range(position.shape[0]):
#             pt1 = (int(position[i][0]), int(position[i][1]))
#             pt2 = (int(position[(i+1) % 4][0]), int(position[(i+1) % 4][1]))
#             cv2.rectangle(image, pt1, pt2, (0, 0, 255), 2)
#     cv2.imwrite('C:\\Users\\Max\\Desktop\\out.jpg', image)
    
    for i,item in enumerate(out):
        text = item['text']
        # 读取付款人
        # 如果当前内容和'深圳市购机汇网络有限公司'至少有八个字符相同，则可认定他们相同
        if len(set(text) & set(payer))>=8:
            invoice.payer = payer
        # 读取日期
        # 如果当前内容是是开票日期，则下一个大概率是日期
        if '开票日期'in text:
            if isDate(out[i+1]['text']):
                invoice.date = out[i+1]['text']
        # 如果当前内容包含2016年，则这个是日期
        if '2016年'in text:
            substring = '2016年'
            index = text.find(substring)
            if index != -1:
                substring_after = text[index + len(substring):]
            invoice.date = substring + substring_after
        # 如果当前内容符合日期格式，则这个是日期
        if isDate(text):
            invoice.date = text
        # 读取金额
        #如果当前内容包含小写
        if '小写' in text:
            #如果当前内容包含 . 则说明小写和金额在同一个内容里
            if '.'in text:
                substring = '小写'
                index = text.find(substring)
                if index != -1:
                    substring_after = text[index+4:]
                invoice.money = substring_after
            #如果当前内容不含 . 则在当前内容的后四个内容里找含 . 的内容
            else:
                for j in range(4):
                    tmp = str(out[i+j]['text'])
                    if '.' in tmp:
                        if tmp[0].isdigit():
                            invoice.money = tmp
                        else:
                            invoice.money = tmp[1:]
    print(invoice.payer,invoice.date,invoice.money)

In [161]:
#把所有的b组图片都识别一次，输出结果
success_num = 0
max_num = 201
index = 0
while index <= max_num:
    if index%10 == 6:
        index += 1
        continue
    print(index,' ')
    ocr_getinfo(index)
    index += 1

0  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
1  
深圳市购机汇网络有限公司 2016年06月12日 1417.00
2  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
3  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
4  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
5  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
7  
深圳市购机汇网络有限公司 2016年06月12日 2987.18
8  
深圳市购机汇网络有限公司 2016年06月12日 
9  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
10  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
11  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
12  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
13  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
14  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
15  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
17  
深圳市购机汇网络有限公司 2016年06月12日 2987.18
18  
深圳市购机汇网络有限公司 2016年06月12日 2987.18
19  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
20  
深圳市购机汇网络有限公司 2016年06月12日 3495.0O
21  
  
22  
  
23  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
24  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
25  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
27  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
28  
深圳市购机汇网络有限公司 2016年06月12日 2987.18
29  
深圳市购机汇网络有限公司 2016年06月12日 2987.18
30  
深圳市购机汇网络有限公司 2016年06月12日 3495.00
31  
深圳市购机汇网

In [57]:
#二维码识别结果的返回格式
class Qrcode_return:
    # 是否识别出二维码、发票代码、发票编号、税前金额、日期
    qrcode_exist: bool
    invoice_code: str
    invoice_num:str
    invoice_amount:str
    invoice_date:str
    def __init__(self,qrcode_exist,invoice_code="",invoice_num="",invoice_amount="",invoice_date=""):
        self.qrcode_exist = qrcode_exist
        self.invoice_code = invoice_code
        self.invoice_num = invoice_num
        self.invoice_amount = invoice_amount
        self.invoice_date = invoice_date
    
def qrcode_getinfo(index):
    image = Image.open('C:\\Users\\Max\\Desktop\\发票-大数据集\\发票-大数据集\\b\\b'+str(index)+'.jpg')
    # 竖图旋转成横图
    width, height = image.size
    if width < height:
        image = image.transpose(method=Image.ROTATE_90)
    # 裁剪出二维码部分
    width, height = image.size
    left = 0
    top = height // 10
    right = width // 4
    bottom = height * 2/5
    cropped_image = image.crop((left, top, right, bottom))
    # 将 PIL 图像转换为 OpenCV 图像数组
    image_array = np.array(cropped_image)
    # 将图像数组转换为灰度图像
    gray_image = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
    # 使用 Otsu's 二值化方法
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    cv2.imwrite('C:\\Users\\Max\\Desktop\\发票-大数据集\\发票-大数据集\\c\\c'+str(index)+'.jpg',binary_image)
    # 调试用：显示二值化图片
#     cv2.imshow("image",binary_image)  
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    # 使用pyzbar解码二维码
    # 二值化图像
    qrcodes = pyzbar.decode(binary_image,symbols=[pyzbar.ZBarSymbol.QRCODE])
    if len(qrcodes)==0:
        qrcode_return = Qrcode_return(False)
        return qrcode_return;
    else:
        for qrcode in qrcodes:
            string = qrcode.data.decode('utf-8')
        info = string.split(',')
        qrcode_return = Qrcode_return(True,info[2],info[3],info[4],info[5])
        return qrcode_return;

In [163]:
#调试用，检查b3的返回结果
x = qrcode_getinfo(3)
print(x.invoice_code)
print(x.invoice_num)
print(x.invoice_amount)
print(x.invoice_date)

4400154130
12269577
2987.18
20160612
